<a href="https://colab.research.google.com/github/epidemia244/epidemia/blob/main/EpidemIA_Monitoramento_por_IA_de_surtos_de_doen%C3%A7as_clim%C3%A1ticas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importa as bibliotecas necessárias
import requests
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import folium
from folium.plugins import FloatImage
import geopandas as gpd
from IPython.display import display

# Função para obter dados climáticos de uma cidade (via Open-Meteo API ou similar)
def obter_dados_climaticos(cidade):
    coordenadas = {
        'Baldim': (-19.2833, -43.9553),
        'Belo Horizonte': (-19.9167, -43.9345),
        'Betim': (-19.9670, -44.1970),
        'Brumadinho': (-20.2086, -44.3858),
        'Caeté': (-19.8833, -43.6700),
        'Capim Branco': (-19.5475, -44.1306),
        'Confins': (-19.6289, -43.9939),
        'Contagem': (-19.9317, -44.0539),
        'Esmeraldas': (-19.7644, -44.3139),
        'Florestal': (-19.8889, -44.4319),
        'Ibirité': (-20.1294, -44.1089),
        'Igarapé': (-20.0703, -44.2992),
        'Itaguara': (-20.1547, -44.3378),
        'Itatiaiuçu': (-20.0903, -44.3650),
        'Jaboticatubas': (-19.5111, -43.7375),
        'Juatuba': (-19.9444, -44.3458),
        'Lagoa Santa': (-19.6308, -43.9000),
        'Mateus Leme': (-19.9797, -44.4317),
        'Mário Campos': (-20.0458, -44.1208),
        'Matozinhos': (-19.5558, -44.0816),
        'Nova Lima': (-19.9850, -43.8467),
        'Nova União': (-19.6875, -43.5836),
        'Pedro Leopoldo': (-19.6181, -44.0433),
        'Raposos': (-19.9628, -43.8078),
        'Ribeirão das Neves': (-19.7661, -44.0867),
        'Rio Acima': (-20.0875, -43.7875),
        'Rio Manso': (-20.0725, -44.1567),
        'Sabará': (-19.8844, -43.8264),
        'Santa Luzia': (-19.7697, -43.8497),
        'Sarzedo': (-20.0100, -44.2431),
        'São Joaquim de Bicas': (-19.7900, -44.1131),
        'São José da Lapa': (-19.6683, -44.0689),
        'Taquaraçu de Minas': (-19.6350, -43.9860),
        'Vespasiano': (-19.7056, -43.9233)
    }

    lat, lon = coordenadas.get(cidade, (-19.6308, -43.9000))

    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
    resposta = requests.get(url)
    if resposta.status_code == 200:
        dados = resposta.json()['current_weather']
        return {
            'temperatura': dados['temperature'],
            'vento': dados['windspeed'],
            'umidade': 70  # Valor fictício
        }
    else:
        raise Exception("Erro ao obter dados climáticos")

# Dados reais de densidade populacional
densidades_reais = {
    'Baldim': 35,
    'Belo Horizonte': 7200,
    'Betim': 2000,
    'Brumadinho': 250,
    'Caeté': 500,
    'Capim Branco': 100,
    'Confins': 120,
    'Contagem': 4000,
    'Esmeraldas': 300,
    'Florestal': 100,
    'Ibirité': 1000,
    'Igarapé': 850,
    'Itaguara': 150,
    'Itatiaiuçu': 120,
    'Jaboticatubas': 55,
    'Juatuba': 800,
    'Lagoa Santa': 180,
    'Mateus Leme': 700,
    'Mário Campos': 300,
    'Matozinhos': 450,
    'Nova Lima': 300,
    'Nova União': 50,
    'Pedro Leopoldo': 250,
    'Raposos': 250,
    'Ribeirão das Neves': 2700,
    'Rio Acima': 90,
    'Rio Manso': 80,
    'Sabará': 900,
    'Santa Luzia': 1600,
    'Sarzedo': 500,
    'São Joaquim de Bicas': 700,
    'São José da Lapa': 400,
    'Taquaraçu de Minas': 250,
    'Vespasiano': 1200
}

# Função para obter densidade populacional real
def obter_densidade_populacional(cidade):
    return densidades_reais.get(cidade, 1000)

# Dados simulados mais variados
X = pd.DataFrame({
    'temperatura': [20, 22, 25, 28, 30, 32, 35, 37, 23, 26, 29, 33, 31, 27],
    'vento':       [3, 5, 4, 6, 10, 7, 8, 6, 2, 5, 7, 9, 8, 4],
    'umidade':     [60, 70, 65, 75, 85, 90, 88, 92, 67, 73, 80, 89, 85, 76],
    'densidade':   [800, 1000, 1500, 6000, 8000, 9000, 7000, 8500, 1200, 5000, 7500, 8800, 8200, 6400]
})
y = [0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]

# Pipeline com padronização e modelo
pipeline = make_pipeline(StandardScaler(), LogisticRegression(C=1.0, solver='liblinear'))

# Validação cruzada para suavizar predições
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
preds_proba = cross_val_predict(pipeline, X, y, cv=kfold, method='predict_proba')

# Ajusta modelo final com todos os dados
treinador = make_pipeline(StandardScaler(), LogisticRegression(C=1.0, solver='liblinear'))
treinador.fit(X, y)

# Função de avaliação de risco
def avaliar_risco_epidemia(cidade):
    clima = obter_dados_climaticos(cidade)
    densidade = obter_densidade_populacional(cidade)
    dados = clima
    dados['densidade'] = densidade
    entrada = pd.DataFrame([dados])
    probabilidade = treinador.predict_proba(entrada)[0][1]
    risco_escala_100 = round(probabilidade * 100, 1)
    return risco_escala_100

# Geração do mapa com Folium usando um GeoJSON externo
def gerar_mapa_risco():
    cidades = [
        'Baldim', 'Belo Horizonte', 'Betim', 'Brumadinho', 'Caeté', 'Capim Branco',
        'Confins', 'Contagem', 'Esmeraldas', 'Florestal', 'Ibirité', 'Igarapé',
        'Itaguara', 'Itatiaiuçu', 'Jaboticatubas', 'Juatuba', 'Lagoa Santa',
        'Mateus Leme', 'Mário Campos', 'Matozinhos', 'Nova Lima', 'Nova União',
        'Pedro Leopoldo', 'Raposos', 'Ribeirão das Neves', 'Rio Acima', 'Rio Manso',
        'Sabará', 'Santa Luzia', 'Sarzedo', 'São Joaquim de Bicas',
        'São José da Lapa', 'Taquaraçu de Minas', 'Vespasiano'
    ]

    geojson_path = 'cidades_mg.json'
    geo_df = gpd.read_file(geojson_path)

    dados_mapa = []
    for cidade in cidades:
        risco = avaliar_risco_epidemia(cidade)
        dados_mapa.append({'cidade': cidade, 'risco': risco})

    df_risco = pd.DataFrame(dados_mapa)
    geo_df = geo_df.merge(df_risco, left_on='name', right_on='cidade', how='left')

    mapa = folium.Map(
        width='100%',
        height='100%',
        location=[-19.9317, -44.0539],
        tiles='cartodb positron',
        zoom_start=9
    )

    folium.Choropleth(
        geo_data=geo_df.dropna(subset=['risco']).to_json(),
        name='choropleth',
        data=geo_df.dropna(subset=['risco']),
        columns=['name', 'risco'],
        key_on='feature.properties.name',
        fill_color='YlOrRd',
        fill_opacity=1,
        line_opacity=1,
        nan_fill_color='transparent',
        legend_name='RISCO DE EPIDEMIA'
    ).add_to(mapa)

    for _, row in geo_df.dropna(subset=['risco']).iterrows():
        tooltip_text = f"{row['name']}: {row['risco']}% de risco" if pd.notnull(row['risco']) else f"{row['name']}: sem dados"
        folium.GeoJson(
            row['geometry'],
            name=row['name'],
            tooltip=tooltip_text,
            style_function=lambda feature: {
                'color': 'black',
                'weight': 0.6,
                'fillOpacity': 0,
            }
        ).add_to(mapa)

    for _, row in geo_df.dropna(subset=['risco']).iterrows():
        cidade = row['name']
        clima = obter_dados_climaticos(cidade)
        densidade = obter_densidade_populacional(cidade)

        popup_text = (
            f"<b>{cidade.upper()}</b><br><br>"
            f"<b>Risco de epidemia:</b> {row['risco']}%<br>"
            f"<b>Temperatura:</b> {clima['temperatura']} °C<br>"
            f"<b>Umidade:</b> {clima['umidade']}%<br>"
            f"<b>Vento:</b> {clima['vento']} km/h<br>"
            f"<b>Densidade populacional:</b> {densidade} hab/km²"
        )

        folium.GeoJson(
            row['geometry'],
            name=cidade,
            tooltip=f"<b>{cidade}:</b> {row['risco']}%",
            popup=folium.Popup(popup_text, max_width=300),
            style_function=lambda feature: {
                'color': 'black',
                'weight': 0.6,
                'fillOpacity': 0,
            }
        ).add_to(mapa) # Adiciona o popup ao mapa

    FloatImage('https://i.postimg.cc/T1tGM5m2/rosa-dos-ventos.png', bottom=8, left=2).add_to(mapa) # Adiciona a rosa dos ventos ao mapa
    FloatImage('https://i.postimg.cc/bJtcVFYT/logo-redimensionada.png', bottom=8, left=84).add_to(mapa) # Adiciona a logo da EpidemIA ao mapa

    folium.LayerControl().add_to(mapa)
    display(mapa)

# Exemplo de uso
gerar_mapa_risco()